# Amazon SageMaker XGBoost and Hyperparameter Tuning for Automobile Insurance Fraud Detection
_**Supervised Learning with Gradient Boosted Trees: A Binary Prediction Problem With Unbalanced Classes**_

---

---

Kernel `Python 3 (Data Science)` works well with this notebook.

## Contents

1. [Background](#Background)
1. [Prepration](#Preparation)
1. [Data Downloading](#Data_Downloading)
1. [Data Transformation](#Data_Transformation)
1. [Setup Hyperparameter Tuning](#Setup_Hyperparameter_Tuning)
1. [Launch Hyperparameter Tuning](#Launch_Hyperparameter_Tuning)
1. [Analyze Hyperparameter Tuning Results](#Analyze_Hyperparameter_Tuning_Results)
1. [Deploy The Best Model](#Deploy_The_Best_Model)
1. [Evaluation](#Evaluation)


---

## Background
Direct marketing, either through mail, email, phone, etc., is a common tactic to acquire customers.  Because resources and a customer's attention is limited, the goal is to only target the subset of prospects who are likely to engage with a specific offer.  Predicting those potential customers based on readily available information like demographics, past interactions, and environmental factors is a common machine learning problem.

This notebook will train a model which can be used to predict if a customer will enroll for a term deposit at a bank, after one or more phone calls. Hyperparameter tuning will be used in order to try multiple hyperparameter settings and produce the best model.

We will use SageMaker Python SDK, a high level SDK, to simplify the way we interact with SageMaker Hyperparameter Tuning.

---

## Preparation

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as SageMaker training.
- The IAM role used to give training access to your data. See SageMaker documentation for how to create these.

In [9]:
import sagemaker
import boto3
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.inputs import TrainingInput

import numpy as np  # For matrix operations and numerical processing
import pandas as pd  # For munging tabular data
import os
import awswrangler as wr

region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)

sagemaker_client = boto3.Session().client("sagemaker")
sagemaker_role = sagemaker.get_execution_role()

s3_client = boto3.client("s3", region_name=region)
sagemaker_session = sagemaker.session.Session(boto_session=boto_session, sagemaker_client=sagemaker_client)

In [2]:
%store -r
%store

Stored variables and their in-db values:
bucket                              -> 'sagemaker-us-east-1-875692608981'
claims_fg_name                      -> 'fraud-detect-demo-claims'
claims_preprocessed                 ->       policy_id  incident_severity  num_vehicles_i
claims_table                        -> 'fraud-detect-demo-claims-1637021687'
clarify_bias_job_1_name             -> 'Clarify-Bias-2021-11-16-01-24-37-110'
clarify_bias_job_2_name             -> 'Clarify-Bias-2021-11-16-02-36-07-707'
clarify_expl_job_name               -> 'Clarify-Explainability-2021-11-16-02-48-36-081'
col_order                           -> ['fraud', 'driver_relationship_spouse', 'num_insur
customers_fg_name                   -> 'fraud-detect-demo-customers'
customers_preprocessed              ->       policy_id  customer_age  customer_education 
customers_table                     -> 'fraud-detect-demo-customers-1637021688'
database_name                       -> 'sagemaker_featurestore'
dataset_uri    

In [3]:
#---> create a pandas dataframe from the combine dataset
dataset = wr.s3.read_csv(path=f"s3://{bucket}/{prefix}/data/dataset/", dataset=True)
dataset

,Unnamed: 0,policy_id,driver_relationship_spouse,num_insurers_past_5_years,policy_state_id,vehicle_claim,authorities_contacted_fire,incident_type_collision,incident_severity,injury_claim,...,driver_relationship_other,policy_state_nv,policy_state_ca,num_claims_past_year,collision_type_front,incident_hour,driver_relationship_self,police_report_available,policy_annual_premium,num_witnesses
0,0,29,0,3,0,13664.0,0,0,1,4700.0,...,0,0,0,0,0,9,0,0,2800,0
1,1,3439,0,1,0,38813.0,0,1,2,4100.0,...,0,0,0,0,0,16,1,1,3000,4
2,2,161,0,1,0,21087.0,0,1,0,7800.0,...,0,0,0,0,0,22,1,0,3000,2
3,3,1893,0,2,0,11411.0,0,1,0,48400.0,...,1,0,1,0,0,15,0,0,2950,0
4,4,3676,0,2,0,10689.0,0,1,0,2900.0,...,0,0,0,0,1,20,1,0,3000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,2105,0,1,0,31124.0,0,1,2,5700.0,...,0,0,0,0,0,5,1,1,3000,1
4996,4996,2134,0,1,0,11500.0,0,1,0,28600.0,...,0,0,1,0,1,19,1,0,2700,0
4997,4997,2630,0,1,0,7900.0,0,1,0,9200.0,...,0,0,1,0,0,20,1,0,3000,0
4998,4998,3039,0,1,0,13413.0,0,1,1,4000.0,...,0,0,1,0,1,18,1,0,2800,1


In [4]:
dataset.to_csv("./data/claims_customer.csv")

In [5]:
# split data
train_data, validation_data, test_data = np.split(
    dataset.sample(frac=1, random_state=1729),
    [int(0.7 * len(dataset)), int(0.9 * len(dataset))],
)

# save preprocessed file to s3
pd.concat([train_data["fraud"], train_data.drop(["fraud", "policy_id"], axis=1)], axis=1).to_csv( "data/hpo_train.csv", index=False, header=False)
pd.concat([validation_data["fraud"], validation_data.drop(["fraud", "policy_id"], axis=1)], axis=1).to_csv("data/hpo_validation.csv", index=False, header=False)
pd.concat([test_data["fraud"], test_data.drop(["fraud", "policy_id"], axis=1)], axis=1).to_csv( "data/hpo_test.csv", index=False, header=False)


### Write train, test and validate data to S3

In [6]:
s3_client.upload_file(Filename="data/hpo_train.csv", Bucket=bucket, Key=f"{prefix}/hpo/train/train.csv")
s3_client.upload_file(Filename="data/hpo_validation.csv", Bucket=bucket, Key=f"{prefix}/hpo/validation/validation.csv")
s3_client.upload_file(Filename="data/hpo_test.csv", Bucket=bucket, Key=f"{prefix}/hpo/test/test.csv")
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key=f"{prefix}/hpo/dataset/dataset.csv")

---

## Setup_Hyperparameter_Tuning 
*Note, with the default setting below, the hyperparameter tuning job can take about 30 minutes to complete.*


Now that we have prepared the dataset, we are ready to train models. Before we do that, one thing to note is there are algorithm settings which are called "hyperparameters" that can dramtically affect the performance of the trained models. For example, XGBoost algorithm has dozens of hyperparameters and we need to pick the right values for those hyperparameters in order to achieve the desired model training results. Since which hyperparameter setting can lead to the best result depends on the dataset as well, it is almost impossible to pick the best hyperparameter setting without searching for it, and a good search algorithm can search for the best hyperparameter setting in an automated and effective way.

We will use SageMaker hyperparameter tuning to automate the searching process effectively. Specifically, we specify a range, or a list of possible values in the case of categorical hyperparameters, for each of the hyperparameter that we plan to tune. SageMaker hyperparameter tuning will automatically launch multiple training jobs with different hyperparameter settings, evaluate results of those training jobs based on a predefined "objective metric", and select the hyperparameter settings for future attempts based on previous results. For each hyperparameter tuning job, we will give it a budget (max number of training jobs) and it will complete once that many training jobs have been executed.

In this example, we are using SageMaker Python SDK to set up and manage the hyperparameter tuning job. We first configure the training jobs the hyperparameter tuning job will launch by initiating an estimator, which includes:
* The container image for the algorithm (XGBoost)
* Configuration for the output of the training jobs
* The values of static algorithm hyperparameters, those that are not specified will be given default values
* The type and number of instances to use for the training jobs

In [10]:
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "latest")

xgb = sagemaker.estimator.Estimator(
    container,
    sagemaker_role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/tuning_jobs".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
)

xgb.set_hyperparameters(
    eval_metric="auc",
    objective="binary:logistic",
    rate_drop=0.3,
)

We will tune four hyperparameters in this examples:
* *eta*: Step size shrinkage used in updates to prevent overfitting. After each boosting step, you can directly get the weights of new features. The eta parameter actually shrinks the feature weights to make the boosting process more conservative. 
* *alpha*: L1 regularization term on weights. Increasing this value makes models more conservative. 
* *num_round*: The number of rounds to run the training.
* *max_depth*: Maximum depth of a tree. Increasing this value makes the model more complex and likely to be overfitted. 

In [11]:
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(1, 10),
    "num_round": IntegerParameter(1, 150),
    "alpha": ContinuousParameter(0, 2),
}

Next we'll specify the objective metric that we'd like to tune and its definition, which includes the regular expression (Regex) needed to extract that metric from the CloudWatch logs of the training job. Since we are using built-in XGBoost algorithm here, it emits two predefined metrics: *validation:auc* and *train:auc*, and we elected to monitor *validation:auc* as you can see below. In this case, we only need to specify the metric name and do not need to provide regex. If you bring your own algorithm, your algorithm emits metrics by itself. In that case, you'll need to add a MetricDefinition object here to define the format of those metrics through regex, so that SageMaker knows how to extract those metrics from your CloudWatch logs.

In [12]:
objective_metric_name = "validation:auc"

Now, we'll create a `HyperparameterTuner` object, to which we pass:
- The XGBoost estimator we created above
- Our hyperparameter ranges
- Objective metric name and definition
- Tuning resource configurations such as Number of training jobs to run in total and how many training jobs can be run in parallel.

In [13]:
tuner = HyperparameterTuner(
    xgb, objective_metric_name, hyperparameter_ranges, max_jobs=20, max_parallel_jobs=3
)

## Launch_Hyperparameter_Tuning
Now we can launch a hyperparameter tuning job by calling *fit()* function. After the hyperparameter tuning job is created, we can go to SageMaker console to track the progress of the hyperparameter tuning job until it is completed.

In [14]:
from sagemaker.session import TrainingInput

train_data_uri = f"s3://{bucket}/{prefix}/hpo/train/train.csv"
validation_data_uri = f"s3://{bucket}/{prefix}/hpo/validation/validation.csv"

train_input = TrainingInput(train_data_uri, content_type="csv")
validation_input = TrainingInput(validation_data_uri, content_type="csv")


In [18]:
import time

tuning_job = f"tuning-{time.strftime('%d-%H-%M-%S', time.gmtime())}"

tuner.fit({"train": train_input, "validation": validation_input}, include_cls_metadata=False, job_name=tuning_job)

............................................................................................................................................................................................................................................................................................................................................................................................................!


Let's just run a quick check of the hyperparameter tuning jobs status to make sure it started successfully.

In [19]:
sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
)["HyperParameterTuningJobStatus"]

'Completed'

# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

In [27]:
tuning_job_name=tuning_job
tuning_job_name

'tuning-16-06-20-10-594a'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [28]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
is_minimize = objective["Type"] != "Maximize"
objective_name = objective["MetricName"]

20 training jobs have completed


In [29]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2021, 11, 16, 6, 40, 13, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.8043609857559204},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 11, 16, 6, 44, 29, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:875692608981:training-job/tuning-16-06-20-10-594a-015-bc0bf8e4',
 'TrainingJobName': 'tuning-16-06-20-10-594a-015-bc0bf8e4',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 11, 16, 6, 43, 20, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '0.20463203820441708',
                          'eta': '0.45513216867462436',
                          'max_depth': '1',
                          'num_round': '7'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [30]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 20
{'lowest': 0.5413540005683899, 'highest': 0.8043609857559204}


,alpha,eta,max_depth,num_round,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
5,0.204632,0.455132,1.0,7.0,tuning-16-06-20-10-594a-015-bc0bf8e4,Completed,0.804361,2021-11-16 06:43:20+00:00,2021-11-16 06:44:29+00:00,69.0
19,0.083192,0.401350,4.0,6.0,tuning-16-06-20-10-594a-001-2ee2d464,Completed,0.798874,2021-11-16 06:23:09+00:00,2021-11-16 06:24:12+00:00,63.0
3,1.982098,0.955804,2.0,4.0,tuning-16-06-20-10-594a-017-6c3e3048,Completed,0.796761,2021-11-16 06:47:13+00:00,2021-11-16 06:48:00+00:00,47.0
7,1.380474,0.456242,1.0,31.0,tuning-16-06-20-10-594a-013-99d99148,Completed,0.786090,2021-11-16 06:41:50+00:00,2021-11-16 06:42:47+00:00,57.0
9,1.244244,0.598107,1.0,22.0,tuning-16-06-20-10-594a-011-f75ba445,Completed,0.785273,2021-11-16 06:37:49+00:00,2021-11-16 06:38:55+00:00,66.0
0,0.634725,0.013055,9.0,144.0,tuning-16-06-20-10-594a-020-feaaa44f,Completed,0.776537,2021-11-16 06:51:34+00:00,2021-11-16 06:52:50+00:00,76.0
13,0.819744,0.099724,10.0,32.0,tuning-16-06-20-10-594a-007-fee9e64c,Completed,0.766824,2021-11-16 06:32:57+00:00,2021-11-16 06:34:02+00:00,65.0
1,0.079244,0.660607,1.0,4.0,tuning-16-06-20-10-594a-019-e1ea380d,Completed,0.757467,2021-11-16 06:49:55+00:00,2021-11-16 06:50:59+00:00,64.0
14,0.004537,0.311711,8.0,12.0,tuning-16-06-20-10-594a-006-b997b49e,Completed,0.738556,2021-11-16 06:28:14+00:00,2021-11-16 06:29:13+00:00,59.0
16,0.004537,0.311711,8.0,13.0,tuning-16-06-20-10-594a-004-433dd6d1,Completed,0.735395,2021-11-16 06:27:54+00:00,2021-11-16 06:29:41+00:00,107.0


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [31]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper:
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]


hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [32]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))

## Deploy the best model
Now that we have got the best model, we can deploy it to an endpoint. Please refer to other SageMaker sample notebooks or SageMaker documentation to see how to deploy a model.